<a href="https://colab.research.google.com/github/Vin-itha/Major_Project/blob/main/trainmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Load and Preprocess Dataset (Train, Validation, Test)
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Update paths (make sure you have folders: train, val, test inside DATASET_PATH)
#DATASET_PATH = "/content/drive/MyDrive/Dataset"

train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/new_dataset/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=123
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/new_dataset/validation",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=123
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/new_dataset/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    seed=123
)

# Store class names
class_names = train_ds.class_names
print("Class names:", class_names)

# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


Found 2248 files belonging to 3 classes.
Found 450 files belonging to 3 classes.
Found 450 files belonging to 3 classes.
Class names: ['Bael', 'Betel', 'Crown Flower']


In [10]:
from tensorflow.keras import layers, models
import tensorflow as tf

# Data Augmentation Layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),   # ±10% rotation
    layers.RandomZoom(0.1),       # ±10% zoom
    layers.RandomContrast(0.2),   # contrast variation instead of brightness
])

# Number of classes
num_classes = len(class_names)
print("Classes:", class_names)

# Build CNN Model
model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),

    data_augmentation,
    # layers.Rescaling(1./255),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.GlobalAveragePooling2D(),  # smaller than Flatten
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Classes: ['Bael', 'Betel', 'Crown Flower']


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_7 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,595 (400.76 KB)

 Trainable params: 102,147 (399.01 KB)

 Non-trainable params: 448 (1.75 KB)

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stop, checkpoint]
)


Epoch 1/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 394s 6s/step - accuracy: 0.9549 - loss: 0.1492 - val_accuracy: 0.5422 - val_loss: 2.2852
Epoch 2/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 434s 5s/step - accuracy: 0.9357 - loss: 0.1752 - val_accuracy: 0.6111 - val_loss: 1.8466
Epoch 3/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 447s 6s/step - accuracy: 0.9638 - loss: 0.1176 - val_accuracy: 0.4800 - val_loss: 3.0505
Epoch 4/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 440s 5s/step - accuracy: 0.9701 - loss: 0.0966 - val_accuracy: 0.6378 - val_loss: 1.1602
Epoch 5/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 444s 6s/step - accuracy: 0.9480 - loss: 0.1239 - val_accuracy: 0.4800 - val_loss: 2.5386
Epoch 6/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 443s 6s/step - accuracy: 0.9600 - loss: 0.1190 - val_accuracy: 0.5667 - val_loss: 1.9282
Epoch 7/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 387s 5s/step - accuracy: 0.9594 - loss: 0.1416 - val_accuracy: 0.5244 - val_loss: 4.9570
Epoch 8/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 441s 5s/step - accuracy: 0.9656 - loss: 0.1007 - val_accuracy: 0.5444 - v

In [15]:
# Evaluate on Test Set
test_loss, test_acc = model.evaluate(test_ds)
print(f"✅ Test Accuracy: {test_acc:.2%}")


15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 971ms/step - accuracy: 0.5534 - loss: 1.5508
✅ Test Accuracy: 54.89%


In [18]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
from google.colab import files

# Load your trained model (Only once — won't retrain)
model = keras.models.load_model('/content/best_model.keras')

# Define class names (match order during training)
class_names = ['bael', 'betel', 'crown flower']

# Upload new image
uploaded = files.upload()  # upload e.g., test1.jpg

# Replace filename below with the uploaded file (or detect dynamically)
image_path = list(uploaded.keys())[0]  # auto-get filename

# Preprocess image
img = tf.keras.utils.load_img(image_path, target_size=(224, 224))
img_array = tf.keras.utils.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, 0)

# Predict
predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions[0])]
confidence = np.max(predictions[0]) * 100

print(f"✅ Predicted class: {predicted_class}")
print(f"🔍 Confidence: {confidence:.2f}%")


Saving test3.jpeg to test3.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
✅ Predicted class: betel
🔍 Confidence: 99.45%
